TRAINING MODEL


In [1]:
import pandas as pd
import numpy as np
import torch
import os
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split


In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Set fixed size for vgg model
    transforms.ToTensor(),           # Convert to [3, 224, 224] Tensor-> array conversion
    transforms.Normalize(            # Normalize pixel values
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

In [3]:
directory=r"mask_data"
categories=['with_mask','without_mask']
data=[]
label=[]

for category in categories:
    path=os.path.join(directory , category)
    for img in os.listdir(path):
        img_path=os.path.join(path , img)
        image=Image.open(img_path).convert('RGB') #TO PIL(pytorch prefer RGB whereas cv2 prefer BGR)
        image=transform(image) 
        
        data.append(image)                 #non numeric values
        label.append(category)           #non numeric values
        
label

c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


['with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with_mask',
 'with

In [4]:
for i in range (len(label)):
    if label[i] =='with_mask':
        label[i]=1
        
    else:
        label[i]=0
label_tensor = torch.tensor(label, dtype=torch.long)
data_tensor = torch.stack(data)
print(label_tensor)
print (data_tensor)

tensor([1, 1, 1,  ..., 0, 0, 0])
tensor([[[[-0.4568, -0.4397, -0.4226,  ..., -1.5185, -1.5185, -1.5014],
          [-0.6794, -0.5596, -0.4054,  ..., -1.5185, -1.5014, -1.4843],
          [-0.4054, -0.4739, -0.4568,  ..., -1.5185, -1.5185, -1.5014],
          ...,
          [-1.6555, -1.6555, -1.6384,  ..., -1.5699, -1.6042, -1.6042],
          [-1.6555, -1.6727, -1.6727,  ..., -1.6042, -1.5699, -1.5528],
          [-1.6898, -1.6898, -1.6555,  ..., -1.5528, -1.5870, -1.6042]],

         [[ 0.0126,  0.0301,  0.0476,  ..., -1.3529, -1.3704, -1.3880],
          [-0.2150, -0.0924,  0.0651,  ..., -1.3529, -1.3529, -1.3704],
          [ 0.0651, -0.0049,  0.0126,  ..., -1.3529, -1.3704, -1.3880],
          ...,
          [-1.4055, -1.4230, -1.4405,  ..., -1.4580, -1.4930, -1.4930],
          [-1.4055, -1.4230, -1.4230,  ..., -1.4930, -1.4580, -1.4405],
          [-1.4405, -1.4405, -1.4405,  ..., -1.4405, -1.4755, -1.4930]],

         [[ 0.6531,  0.6705,  0.6879,  ..., -1.0724, -1.0724, -1.0898

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data_tensor, label_tensor, test_size=0.2, random_state=42)

In [6]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch

class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [7]:
train_dataset = CustomDataset(X_train, y_train, transform=transform)
test_dataset = CustomDataset(X_test, y_test, transform=transform)

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [9]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
for param in vgg16.features.parameters():
    param.requires_grad=False

X_train.shape

torch.Size([2538, 3, 224, 224])

In [11]:
import torch.nn as nn
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096),  # VGG16's flattened input size
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(0.3),

    nn.Linear(1024, 2)  # Output: 2 classes (mask/no mask)
)

In [12]:
learning_rate = 0.0001
epochs = 10

In [13]:
criterion = nn.CrossEntropyLoss()
import torch.optim as optim
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [14]:
for epoch in range (epochs):
    for batch_features, batch_labels in train_loader:
        
        output=vgg16(batch_features)
        
        loss=criterion(output,batch_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break

c:\Users\goyal\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [15]:
# evaluation on test data
vgg16.eval()#disables layers like Dropout and BatchNorm
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9748031496062992


COMPUTER VISION

In [16]:
import cv2

In [17]:
#detection function
def detect_face_mask(frame):
    vgg16.eval()  # set model to evaluation mode
    with torch.no_grad():
        img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # BGR to RGB
        img_transformed = transform(img_pil).unsqueeze(0)  # add batch dimension
        output = vgg16(img_transformed)
        _, prediction = torch.max(output, 1)
        return prediction.item()  #returns 0 and 1


In [18]:
cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()
    #detection method
    y_pred=detect_face_mask(frame)
    
    # Show prediction
    if y_pred == 1:
        label = "Mask"
        color = (0, 255, 0)#green
    else:
        label="No Mask"
        color=(0, 0, 255)#red
        
    cv2.putText(frame, label, (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)    
    
    cv2.imshow('camera',frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

